In [1]:
import modin

In [2]:
modin.__version__

'0.15.2'

In [3]:
import modin.pandas as mpd
import pandas as pd
from modin.distributed.dataframe.pandas import unwrap_partitions, from_partitions
import haystack
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever
import tqdm
import ray

In [4]:
ray.init(log_to_driver=False)

2022-08-02 09:38:48,823	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.150.0.18',
 'raylet_ip_address': '10.150.0.18',
 'redis_address': '10.150.0.18:6379',
 'object_store_address': '/tmp/ray/session_2022-08-02_09-38-46_405399_30728/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-08-02_09-38-46_405399_30728/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-08-02_09-38-46_405399_30728',
 'metrics_export_port': 45072,
 'gcs_address': '10.150.0.18:46737',
 'node_id': 'e73ce92fe06f3f88397fbab622152627a45743f1c5c95cd58409a8d6'}

In [5]:
!ls ../data

Posts.xml     df_raw	      stackoverflow.com-Posts.7z
df_processed  df_raw.parquet  start.sh


In [6]:
ds = ray.data.read_parquet("../data/df_processed")


Metadata Fetch Progress: 100%|██████████| 5/5 [00:00<00:00, 13.36it/s]


In [7]:
ds

Dataset(num_blocks=32, num_rows=11588225, schema={Id: int64, AcceptedAnswerId: int64, Title: string, Body: string, Tags: string, ViewCount: int64, AnswerCount: int64, CommentCount: int64, Score: int64, CreationDate: timestamp[us], AnswerBody: string})

In [11]:
ds.take(1)

[{'Id': 19419450, 'AcceptedAnswerId': 19419843, 'Title': "How to handle encrypted URL's in rails?", 'Body': 'I am sending email to user, in that email one link is there to redirect that user to rails application. I want that link to be in encrypted form with domain name for example:\n\nhttps://www.domain_name.com?hdurstihnzdfalgfgdfhdrbnhduolsasrtyumyrtyr\n\nwhen user click on this link, he should directly redirect to controller method we specified in that URL which is not visible.\n\nController and methods given in URL may vary according to user.\n\nSo my question is how we can do this in rails. \n\nIf I encrypt controller name, method name and parameter we passed. How routes file come to know where to redirect this URL? How to decrypt this in routes file and redirect internally to decrypted URL?\n', 'Tags': 'ruby,ruby-on-rails-3,ruby-on-rails-3.2,ruby-on-rails-3.1', 'ViewCount': 1900, 'AnswerCount': 2, 'CommentCount': 0, 'Score': 1, 'CreationDate': datetime.datetime(2013, 10, 17, 6, 

In [ ]:
#df = pd.read_parquet("../data/df_processed")

In [ ]:
#df.head()

In [ ]:
#df.shape

In [ ]:
#df.head(5)

In [13]:
def convert_to_document_dict(row):
    d = row.to_dict()
    #d['content']=d['Title'] + "\n" + d['Body']
    
    d['content']=d['Title']
    d['id']= d['Id']
    d['Tags'] = d['Tags'].split(",")
    del d['Id']
    d_doc = haystack.schema.Document.from_dict(d)
    
    return d_doc


def fetch_results(query:str, retriever, top_k=10):
    candidate_documents = retriever.retrieve(
        query=query,
        top_k=10,
    #filters={"year": ["2015", "2016", "2017"]}
    )

    for doc in candidate_documents:
        print (doc.meta['Title'], doc.score, doc.id )
        
        
def index_df(df, document_store):
    docs = df.apply(convert_to_document_dict, axis=1).to_numpy().flatten().tolist()
    
    document_store.write_documents(docs, batch_size=1_000)
    
def index_individual_df(df, document_store):
    
    docs = df.apply(convert_to_document_dict, axis=1)\
            .apply(lambda x: document_store.write_documents ( [doc]))
    
    
def transform_batch(df: pd.DataFrame) -> pd.DataFrame:
    return len(df)

In [ ]:
#convert_to_document_dict ( df.iloc[0] )

In [ ]:
#df.head(5).apply(convert_to_document_dict, axis=1)

In [ ]:

document_store = ElasticsearchDocumentStore(host="np-database.c.np-training.internal"
                                            , index="stackoverflow", port=9200)


retriever = BM25Retriever(document_store)


In [ ]:
document_store.delete_documents()

In [ ]:
def apply_func(df):
    print ( len (df))



In [ ]:
df.apply(lambda s: apply_func(s.to_frame()), axis=1)

In [ ]:
@ray.remote
def index_df_with_remote(partition):
    df_subset = from_partitions( [ partitions ] ,axis=0)
      
    #docs = df_subset.apply(convert_to_document_dict, axis=1).to_numpy().flatten().tolist()
    
    #return  len (docs)
     
    return 5

In [ ]:
partitions = unwrap_partitions(df, axis=0, get_ip=True)


In [ ]:
futures = [index_df_with_remote.remote(partition) for partition in partitions]


In [ ]:
print(ray.get(futures)) # [0, 1, 4, 9]


In [ ]:
?document_store.write_documents

In [ ]:
index_df(df, document_store)

In [ ]:
# index_individual_df(df, document_store)

In [ ]:
!ls

In [ ]:
#partitions = unwrap_partitions(df.head(500))



In [ ]:

# partitions = unwrap_partitions(df, axis=0, get_ip=True)

# for i in tqdm.tqdm (range(len (partitions))):
    
#     df_subset = from_partitions(partitions[i:i+1],axis=0)
    
#     docs = df_subset.apply(convert_to_document_dict, axis=1).to_numpy().flatten().tolist()
    
#     document_store.write_documents(docs)

In [ ]:
document_store.write_documents(docs)

In [ ]:
document_store.get_document_count()

In [ ]:
fetch_results(query="encrypted URL", retriever=retriever)